In [1]:
import numpy as np
import matplotlib.pyplot as plt
import uproot

In [90]:
# Constants for the fiducial cut calculations
scoringPlaneZ = 239.9985
ecalFaceZ = 247.932
cell_radius = 5.0

# Project the recoil coordinates from the scoring plane to the ecal face.
def projection(Recoilx, Recoily, Recoilz, RPx, RPy, RPz, HitZ):
    x_final = Recoilx + RPx / RPz * (HitZ - Recoilz) if RPz != 0 else 0
    y_final = Recoily + RPy / RPz * (HitZ - Recoilz) if RPy != 0 else 0
    return (x_final, y_final)

# Calculate the Euclidean distance between two points.
def dist(cell, point):
    return np.sqrt((cell[0] - point[0])**2 + (cell[1] - point[1])**2)

# Load cell map from file.
def load_cellMap(filepath):
    cellMap = np.genfromtxt(filepath, dtype={'names': ('x', 'y', 'id'), 'formats': ('f4', 'f4', 'i4')}, usecols=[1,2,0])
    print(f"Loaded detector info from {filepath}")
    return cellMap

# Calculate indices of recoil electron for all events
def find_recoil_electron(pdgID, pz):
    # Initialize indices as -1 (not found or conditions not met)
    r_electron = np.full(len(pdgID), -1, dtype=int)
    
    # Iterate over each event
    for i in range(len(pdgID)):
        # Find indices where particle ID is 11 (electron)
        electron_indices = np.where(pdgID[i] == 11)[0]
        
        # If no electrons or all pz values are zero, continue
        if len(electron_indices) == 0 or np.all(pz[i][electron_indices] == 0):
            continue
        
        # Find the index with the maximum pz among electrons
        # This step filters out all pz == 0 automatically since they can't be the maximum if there's any non-zero pz
        max_pz_index = electron_indices[np.argmax(pz[i][electron_indices])]
        
        # Set the recoil electron index for this event
        r_electron[i] = max_pz_index
    
    return r_electron
    
# Apply the fiducial cut to the given recoil data.
def apply_fiducial_cut(recoilX, recoilY, recoilZ, recoilPx, recoilPy, recoilPz, recoilElectron, cells):
    N = len(recoilElectron)
    f_cut = np.zeros(N, dtype=bool)
    for i in range(N):
        fiducial = False
        recoil_i = recoilElectron[i]
        if recoil_i != -1:
            fXY = projection(recoilX[i][recoil_i], recoilY[i][recoil_i], recoilZ[i][recoil_i], 
                             recoilPx[i][recoil_i], recoilPy[i][recoil_i], recoilPz[i][recoil_i], ecalFaceZ)
            # if not all(val == -9999 for val in [recoilX[i][recoil_i], recoilY[i][recoil_i], recoilZ[i][recoil_i], 
            #                                     recoilPx[i][recoil_i], recoilPy[i][recoil_i], recoilPz[i][recoil_i]]):
            for cell in cells:
                if dist(cell, fXY) <= cell_radius:
                    fiducial = True
                    # print(cell)
                    break
        f_cut[i] = fiducial
    return f_cut

In [103]:
# Load cell information
cells = load_cellMap('/home/j3lly/LDMX-scripts/GraphNet/data/v14/cellmodule.txt')

# Path to the ROOT file
# file_path = '/home/vamitamas/NonFiducialSimu/events_nonfiducial_fullEcal_production.root'
# file_path = '/mnt/d/Repositories/rootfiles/Processed8GeV/v14_8gev_0.1_XCal_total_40.root'
file_path = '/mnt/d/Repositories/rootfiles/Samples8GeV/Ap0.001GeV_sim/mc_v14-8gev-8.0GeV-1e-signal_W_noDecay_sim_run10059_t1699049052.root'

Loaded detector info from /home/j3lly/LDMX-scripts/GraphNet/data/v14/cellmodule.txt


In [104]:
# Open the ROOT file and load branches
with uproot.open(file_path) as file:
    tree = file["LDMX_Events"]
    leaf_pdgID = tree['TargetScoringPlaneHits_signal.pdgID_'].array(library='np')
    leaf_x = tree['TargetScoringPlaneHits_signal.x_'].array(library='np')
    leaf_y = tree['TargetScoringPlaneHits_signal.y_'].array(library='np')
    leaf_z = tree['TargetScoringPlaneHits_signal.z_'].array(library='np')
    leaf_px = tree['TargetScoringPlaneHits_signal.px_'].array(library='np')
    leaf_py = tree['TargetScoringPlaneHits_signal.py_'].array(library='np')
    leaf_pz = tree['TargetScoringPlaneHits_signal.pz_'].array(library='np')

In [105]:
recoilElectron = find_recoil_electron(leaf_pdgID, leaf_pz)

In [106]:
# Apply the fiducial cut
f_cut = apply_fiducial_cut(leaf_x, leaf_y, leaf_z, leaf_px, leaf_py, leaf_pz, recoilElectron, cells)

In [107]:
# Calculate and print statistics
fiducial_events = np.sum(f_cut)
valid_events = np.count_nonzero(recoilElectron != -1)
total_events = len(f_cut)
non_fid_events = valid_events-fiducial_events
ratio = non_fid_events / fiducial_events
print(f"Total Events: {total_events}")
print(f"Electron Events: {valid_events}")
print(f"Fiducial Events: {fiducial_events}")
print(f"Non-Fiducial Events: {non_fid_events}")
print(f"Non-Fiducial/Fiducial Events Ratio: {ratio:.4f}")

Total Events: 20000
Electron Events: 20000
Fiducial Events: 20000
Non-Fiducial Events: 0
Non-Fiducial/Fiducial Events Ratio: 0.0000


In [108]:
# # Plotting the histogram of fiducial vs non-fiducial events
# plt.hist(f_cut.astype(int), bins=[-0.5, 0.5, 1.5], rwidth=0.8) # label=['Non-Fiducial', 'Fiducial'])
# plt.xticks([0, 1], ['Non-Fiducial', 'Fiducial'])
# plt.xlabel('Event Type')
# plt.ylabel('Number of Events')
# plt.title('Fiducial vs Non-Fiducial Events')
# # plt.legend()
# plt.show()

In [56]:
test_arr = np.zeros(3, dtype=int)

In [68]:
test_arr[1]=-1

In [69]:
test_arr

array([ 0, -1,  1])

In [71]:
import numpy as np

def find_recoil_electron(pdgID, pz):
    # Initialize indices as -1 (not found or conditions not met)
    r_electron = np.full(len(pdgID), -1, dtype=int)
    
    # Iterate over each event
    for i in range(len(pdgID)):
        # Find indices where particle ID is 11 (electron)
        electron_indices = np.where(pdgID[i] == 11)[0]
        
        # If no electrons or all pz values are zero, continue
        if len(electron_indices) == 0 or np.all(pz[i][electron_indices] == 0):
            continue
        
        # Find the index with the maximum pz among electrons
        # This step filters out all pz == 0 automatically since they can't be the maximum if there's any non-zero pz
        max_pz_index = electron_indices[np.argmax(pz[i][electron_indices])]
        
        # Set the recoil electron index for this event
        r_electron[i] = max_pz_index
    
    return r_electron

# Example data
pdgID = [np.array([11, 22, 11]), np.array([11, 11]), np.array([22])]
pz = [np.array([1.2, 4, 2.3]), np.array([0.0, 0.0]), np.array([0.0])]

# Find the recoil electrons
recoil_indices = find_recoil_electron(pdgID, pz)
print("Recoil indices:", recoil_indices)


Recoil indices: [ 2 -1 -1]


In [86]:
if 0:
    print(1)